#### Задание
В данном задании вам нужно будет провести анализ данных проведенного A/B-теста.

Данные будут предоставлены в формате csv. Они будут содержать также часть пред экспериментального этапа.

Описание эксперимента:
Есть мобильное приложение. В этом приложении у пользователей есть возможность покупать игровые предметы за реальные деньги. Чтобы стимулировать пользователей их покупать, приложение периодически предлагает пользователям товары - появляется окошко с рекомендацией купить товар. Отдел машинного обучения предложил улучшение для текущего алгоритма выбора рекомендации. Для проверки улучшений алгоритма был проведен A/B тест. Лог его проведения предоставлен в прикрепленном файле.

Метрика: средний доход от пользователя за 1 неделю после первого показа ему рекомендации на 10% (после начала A/B теста время первого показа ищется снова)

Важная информация:
Эксперимент начинается 2023-05-01. Данные есть до 2023-06-01 (но можно завершить раньше, если это позволит оценка длительности)
Вам сказали, что его длительность должна составить 1 месяц.
Все покупки, которые вызваны не влиянием рекомендаций, в этом логе не учитываются

Описание данных:
id_product -  идентификатор продукта, который был рекомендован
is_pay - купил ли пользователь товар
sum_payment - размер платежа (0, если не купил)
city - город, в котором находится пользователь
id_user - пользователь
timestamp - timestamp события
date - дата события

Задачи, которые необходимо решить:
Оценить длительность теста на момент его начала. Сравнить с предложенной. Для оценки необходимо использовать данные с пред экспериментального периода. Посмотреть, есть ли выбросы в данных.
Построить методику расчета целевой метрики. Рассчитать целевую метрику на день окончания теста (рассчитанной в п1) для группы A и B, рассчитать эффект, p_value. Посмотреть, есть ли выбросы в данных.
Рассчитать метрики из п2 по дням и построить их графики.
Принять решение о результате теста - обосновать.

Дополнительно:
При подсчете метрик обработать выбросы в данных - описать почему их можно удалить из теста, если они удаляются. Пожалуйста, не заглядывайте в будущее по данным, если рассчитанная длительность меньше предоставленных данных.


In [13]:
import numpy as np
import pandas as pd
import scipy.stats as sps
from datetime import timedelta
import plotly.graph_objs as go

In [11]:
data = pd.read_csv("/Users/sofia/Downloads/ab_made_4.gzip")

In [14]:
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s') + timedelta(hours=3)
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
pre_group = data[data['group'].isna()]
A = data[data['group'] == 'A']
B = data[data['group'] == 'B']

In [15]:
def duration(k, delta_effect, sigma_1, sigma_2, alpha=0.05, beta=0.2):
    z = sps.norm.ppf(1 - alpha/2) + sps.norm.ppf(1-beta)
    n = (k+1) * z ** 2 * (sigma_1 ** 2 + sigma_2 **2 / k) / (delta_effect ** 2)
    return n

def metric(data, end):
    filtered = data.groupby('id_user').timestamp.min().reset_index(name='min_timestamp')
    filtered['max_timestamp'] = filtered['min_timestamp'] + timedelta(days=7)

    merged = data.merge(filtered, on='id_user')
    merged = merged[(merged['timestamp'] <= merged['max_timestamp']) & (merged['timestamp'] >= merged['min_timestamp'])]
    merged = merged[pd.to_datetime(merged['max_timestamp']).dt.normalize() < end]

    return merged.groupby('id_user').sum_payment.sum().tolist()

def filter_date(data, target_n, start, end):
    date = start + timedelta(days=7)

    while True:
        if date == end:
            return date
        temp_data = data[data['date'] <= date]

        filtered = data.groupby('id_user').timestamp.min().reset_index(name='min_timestamp')
        filtered['max_timestamp'] = filtered['min_timestamp'] + timedelta(days=7)
        filtered = filtered[pd.to_datetime(filtered['max_timestamp']).dt.normalize() <= date]

        if len(np.unique(filtered["id_user"].values)) >= target_n:
            return date

        date = date + timedelta(days=1)

def remove_outliers(data, threshold = 0.001 ):
    lower_bound = data["sum_payment"].quantile(q=threshold)
    upper_bound = data["sum_payment"].quantile(q=1-threshold)
    emission_df = data[(data["sum_payment"] < lower_bound) | (data["sum_payment"] > upper_bound)]

    data.loc[data.index.isin(emission_df.index), "sum_payment"] = \
        data.loc[data.index.isin(emission_df.index), "sum_payment"].apply(lambda x: min(x, upper_bound))

    return data



In [17]:
start = pd.to_datetime('2023-05-01', format='%Y-%m-%d')
end = pd.to_datetime('2023-05-31', format='%Y-%m-%d')

pre_group = remove_outliers(pre_group)
metric_out = metric(pre_group, start)
sigma_1 = sigma_2 = np.std(metric_out)

mean_base = np.mean(metric_out)
effect = 0.1 * mean_base

k = len(np.unique(A['id_user'].values)) / len(np.unique(B['id_user'].values))

target_n = int(duration(k, effect, sigma_1, sigma_2)/2)

print(f"Target size: {target_n}")



Target size: 3601


/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
